Only reading in one email for now

In [1]:
enroncsv = "../experiments/data/enron.csv"
metadataHeaders = '../experiments/data/metadataHeaders.csv'
spam = '../experiments/data/spam.csv'
spamSubjects = '../experiments/data/spamWords.csv'

In [2]:
import pandas as pd

columns = pd.read_csv(metadataHeaders, sep=',').columns.tolist()

In [3]:
df = pd.read_csv(enroncsv, names=columns, sep='|', low_memory=False)

print("-- DONE --")

-- DONE --


In [4]:
removableColumns = pd.read_csv('data/removableColumns.csv', sep=',').columns.tolist()

df.drop(removableColumns, axis=1)

print('-- DONE --')

-- DONE --


In [5]:
listOfEmailsForBagOfWords = df.loc[df['Directory'].str.contains('inbox')]

print('-- DONE --')

-- DONE --


In [6]:
wordsPerEmail = df[pd.notnull(df['Subject'])]
nonSpamSubjects = wordsPerEmail['Subject']
wordsPerEmail = nonSpamSubjects.str.split(' ')
len(wordsPerEmail)

498161

In [7]:
import string
printable = set(string.printable)

def isEnglish(s):
    for x in s:
        if x not in printable:
            return False
    return True

def addToBagOfWords(dictionary, arr):
    for word in arr:
        word = word.lower()
        
        if '=' in word:
            continue
        if '/' in word:
            continue
        if '\\' in word:
            continue
        if '_' in word:
            continue
        if '-' in word:
            continue
        if ':' in word:
            continue
        if '@' in word:
            continue
        if '#' in word:
            continue
        if '$' in word:
            word = '$'

        word = word.replace('.', '')
        word = word.replace(')', '')
        word = word.replace('(', '')
        word = word.replace('&', '')
        word = word.replace('\'', '')
        word = word.replace('\"', '')
        word = word.replace(',', '')
        word = word.replace('[', '')
        word = word.replace(']', '')
        word = word.replace('{', '')
        word = word.replace('}', '')
        word = word.replace(';', '')

        if word == '':
            continue

        if word in dictionary:
            dictionary[word] = dictionary[word] + 1
        else:
            dictionary[word] = 1
            
    return dictionary

In [8]:
def addToDictionary(dictionary, arr):
    for index, subject in arr.iteritems():
        dictionary = addToBagOfWords(dictionary, subject)
                
    return dictionary

In [9]:
dictionary = {}

dictionary = addToDictionary(dictionary, wordsPerEmail)

len(dictionary)

39519

In [10]:
spamSubjects = pd.read_csv(spam, names=['Subject'], sep=',')

In [11]:
wordsPerEmail = spamSubjects['Subject'].str.split(' ')

dictionary = addToDictionary(dictionary, wordsPerEmail)
        
len(dictionary)

45090

In [12]:
dictionaryList = []
for key in dictionary:
    dictionaryList.append([key, dictionary[key]])
    
dictionaryList = pd.DataFrame(dictionaryList, columns=["Word", "Occurances"])
len(dictionaryList)

45090

We will only use words that occurred more than 30 times in our bag of words as other words would almost never match up anyway

In [13]:
dictionaryList = dictionaryList.loc[dictionaryList['Occurances'] > 30].reset_index(drop=True)
len(dictionaryList)

5751

In [14]:
def addBagToInputOutput(inputItems, outputItems, arr):
    for index, subject in inputItems.iteritems():
        posBagOfWords = {}
        posBagOfWords = addToBagOfWords(posBagOfWords, subject)

        listOfIn = []
        for key in posBagOfWords:
            listOfIn.append(key)

        inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()

        arr.append([np.array(inputBag), np.array(outputItems)])
        
    return arr
    

### Create training data

In [15]:
import matplotlib.pyplot as plt
import numpy as np

training = []

NON_SPAM_SAMPLES = 5000
SPAM_SAMPLES = 5000

trainingPos = df[pd.notnull(df['Subject'])]
trainingPos = trainingPos.sample(NON_SPAM_SAMPLES)
trainingPos = trainingPos['Subject']

trainingPosSplit = trainingPos.str.split(' ')

training = addBagToInputOutput(trainingPosSplit, [0], training)
    
trainingNeg = spamSubjects.sample(SPAM_SAMPLES)
trainingNeg = trainingNeg['Subject']

trainingNegSplit = trainingNeg.str.split(' ')
    
training = addBagToInputOutput(trainingNegSplit, [1], training)

training = np.array(training)
np.random.shuffle(training)

len(training)

10000

Lets set up a neuarl network with neurons on each layer 

    input - 5558 + bias
    hidden layer 1 - 200 + bias
    hidden layer 2 - 20 + bias
    output - 1
    
The learing rate is set to 0.001
The netword is bound to (-2, 2)

In [16]:
from mlpy.numberGenerator.bounds import Bounds
from mlpy.neuralNetwork.feedForwardNeuralNetwork import NeuralNetwork
from mlpy.neuralNetwork.structure.layer import Layer

l_rate = 0.001
bounds = Bounds(-2, 2)

inputLayer = Layer(bounds, size = len(training[0][0]), prev = None, l_rate = l_rate, bias = True, label = "Input layer")
hiddenLayer = Layer(bounds, size = 300, prev = inputLayer, l_rate = l_rate, bias = True, label = "Hidden layer")
hiddenLayer2 = Layer(bounds, size = 30, prev = hiddenLayer, l_rate = l_rate, bias = True, label = "Hidden layer 2")
outputLayer = Layer(bounds, size = len(training[0][1]), prev = hiddenLayer2, l_rate = l_rate, bias = False, label = "Output layer")

fnn = NeuralNetwork()
fnn.appendLayer(inputLayer)
fnn.appendLayer(hiddenLayer)
fnn.appendLayer(hiddenLayer2)
fnn.appendLayer(outputLayer)

group_training = np.array([input[0] for input in training])
group_target = np.array([output[1] for output in training])

errors = []

We will run training over 4000 iterations and output the mean error every 200 iterations.

In [17]:
ITERATIONS = 8000

print("Starting..")
for i in range(ITERATIONS):
    mod = i % len(training)
    in_out = training[mod]
    result = fnn.fire(group_training)
    error = fnn.backPropagation(group_target)
    
    if i % 50 == 0:
        print(str(np.round(i/ITERATIONS*100)) + '%\t', error.mean())
    
print("-- DONE --")

Starting..
0.0%	 -0.12995725726515203
1.0%	 -0.21724487662987976
1.0%	 -0.18667304135646745
2.0%	 -0.154810128458398
2.0%	 -0.13319424540570068
3.0%	 -0.11482310680891214
4.0%	 -0.10200137026296527
4.0%	 -0.09131738005633179
5.0%	 -0.08177446592544309
6.0%	 -0.07356384744998444
6.0%	 -0.06472169480361051
7.0%	 -0.056788084025721296
8.0%	 -0.05390726930064169
8.0%	 -0.05214460293985674
9.0%	 -0.04926056579967013
9.0%	 -0.046819271107435896
10.0%	 -0.04360912417521026
11.0%	 -0.03953216966816548
11.0%	 -0.034343089128355986
12.0%	 -0.028134008620796118
12.0%	 -0.022479422292637174
13.0%	 -0.018752070351201765
14.0%	 -0.014828378816415114
14.0%	 -0.009150373463211663
15.0%	 -0.004839548813960415
16.0%	 -0.003110097369714201
16.0%	 -0.002767333887769694
17.0%	 -0.002777686060159787
18.0%	 -0.002862039866603216
18.0%	 -0.0029086166534290085
19.0%	 -0.0028686689112518357
19.0%	 -0.0027967610791959138
20.0%	 -0.0027035285623649334
21.0%	 -0.002669235427740258
21.0%	 -0.002716969116087496
22.0

### Create testing data

In [18]:
testing = []

TEST_NON_SPAM_SAMPLES = 5000
TEST_SPAM_SAMPLES = 5000

trainingPos = df[pd.notnull(df['Subject'])]
trainingPos = trainingPos.sample(TEST_NON_SPAM_SAMPLES)
trainingPos = trainingPos['Subject']

trainingPosSplit = trainingPos.str.split(' ')

testing = addBagToInputOutput(trainingPosSplit, [0], testing)
    
trainingNeg = spamSubjects.sample(TEST_SPAM_SAMPLES)
trainingNeg = trainingNeg['Subject']

trainingNegSplit = trainingNeg.str.split(' ')

testing = addBagToInputOutput(trainingNegSplit, [1], testing)

testing = np.array(testing)
np.random.shuffle(testing)

len(testing)

10000

Testing the model we will take the test data and use it as an measure of performace. The following will be output:

1. The Classification Accuracy
2. The Non-spam Classsification accuracy
3. The Spam Classification accuracy

In [19]:
correct = 0
spamCorrect = 0
nonSpamCorrect = 0
for i in range(len(testing)):
    in_out = testing[i]
    result = fnn.fire(np.array([in_out[0]]))
    
    target = in_out[1][0]
    result = np.round(result[0][0])

    if result == target:
        correct += 1
        if target == 1:
            spamCorrect += 1
        else:
            nonSpamCorrect += 1
        
        
print("Classification accuracy: ", correct / len(testing))
print("Non spam classification accuracy: ", nonSpamCorrect / TEST_NON_SPAM_SAMPLES)
print("Spam classification accuracy: ", spamCorrect / TEST_SPAM_SAMPLES)

Classification accuracy:  0.8731
Non spam classification accuracy:  0.8638
Spam classification accuracy:  0.8824


# Profiling:

### Lets apply the classifier to a subset of the data to see what it classifies as spam

In [25]:
dfClassify = df.drop(['Filename', 'Person', 'Directory', 'Message-ID', 'Content-Transfer-Encoding', 'Content-Type', 'Date', 'X-FileName', 'X-To', 'X-bcc', 'Cc', 'X-cc', 'X-Folder', 'X-Origin', 'Time', 'Bcc', 'X-From', 'Attendees', 'Re', 'Mime-Version'], axis=1)

dfClassify = dfClassify[pd.notnull(df['Subject'])]
dfClassify = dfClassify.sample(len(dfClassify))

def classify(arr):
    classifySplit = arr.split(' ')
    
    posBagOfWords = {}
    posBagOfWords = addToBagOfWords(posBagOfWords, classifySplit)

    listOfIn = []
    for key in posBagOfWords:
        listOfIn.append(key)

    inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()
    
    result = fnn.fire(np.array([np.array(inputBag)]))[0][0]
    
    return result

dfClassify['Classification'] = dfClassify['Subject'].apply(classify)
    
dfClassify.sort_values(['Classification'], ascending=[0])

,From,To,Subject,Classification
206775,edirectnetwork@optin-offers.net,pmims@enron.com,Get Your FREE 10-Cup Coffeemaker today!,1.000000e+00
150630,ipay.debt-return-11-7-10825134-x3pacl23neeotqb...,don.baughman@enron.com,Take Control of Your Debt Today!,1.000000e+00
206688,consumerinfo@opt29.naviantnetwork.net,pmims@enron.com,Free Credit Report: Get yours now,1.000000e+00
125484,justatip@processrequest.com,phillip.platter@enron.com,Your guaranteed winning scratch card is right ...,1.000000e+00
206738,wincash@play4keeps.com,pmims@enron.com,Free Bracelet - Your Elite Member Reward,1.000000e+00
150755,wincash@play4keeps.com,don.baughman@enron.com,Free Bracelet - Your Elite Member Reward,1.000000e+00
20712,dert55@arabia.com,conaribuaion.1345@mailman.enron.com,"Buying a car, fix your credit NOW.",1.000000e+00
157773,mark.confer@enron.com,"lisa.gang@enron.com, julie.sarnowski@enron.com...",Want to make sure people are aware,1.000000e+00
179202,leads@mail.ru,undisclosed.recipients@mailman.enron.com,Lower your monthly payments and GET CASH Today...,1.000000e+00
206214,irene@m-ul.com,pmims@enron.com,Refinance your home now!,1.000000e+00


In [26]:
dfClassify[dfClassify['From'] == "wincash@play4keeps.com"]

,From,To,Subject,Classification
206461,wincash@play4keeps.com,pmims@enron.com,Get Your FREE Credit Report!,1.000000
150712,wincash@play4keeps.com,don.baughman@enron.com,"Click here to claim your share of $7,000,000,0...",0.999950
206774,wincash@play4keeps.com,pmims@enron.com,Start 2002 with a NEW Car!!,0.990906
206614,wincash@play4keeps.com,pmims@enron.com,"Click here to claim your share of $7,000,000,0...",0.999950
150755,wincash@play4keeps.com,don.baughman@enron.com,Free Bracelet - Your Elite Member Reward,1.000000
392934,wincash@play4keeps.com,plucci@ect.enron.com,Here?s your $200 FREE GAS!!,1.000000
150767,wincash@play4keeps.com,don.baughman@enron.com,Is there pornography on your computer? FREE Sy...,0.999955
206595,wincash@play4keeps.com,pmims@enron.com,Refinance Without Perfect Credit,1.000000
206310,wincash@play4keeps.com,pmims@enron.com,Is there pornography on your computer? FREE Sy...,0.999955
206841,wincash@play4keeps.com,pmims@enron.com,Never Lose Another Mobile Call,0.118899


Lets group by the sender and see which mailing address sent the most spam

In [94]:
groupedClassify = dfClassify[dfClassify['Classification'] > 0.90]
groupedClassify = groupedClassify.loc[groupedClassify['From'].str.contains('enron.com', regex=True) == False]

In [95]:
groupedClassify = groupedClassify.groupby(['From']).agg(['count', 'mean'])
groupedClassify = groupedClassify.reset_index()

In [96]:
groupedClassify.sort_values([('Classification', 'count')], ascending=[0]).head()

From Classification          
                                                  count      mean
1367                 fool@motleyfool.com            103  0.978953
2796  owner-eveningmba@haas.berkeley.edu             96  0.979132
2669                   news@real-net.net             89  0.995501
1206              enerfax1@bellsouth.net             89  0.999506
3734                   vkaminski@aol.com             76  0.957927